In [1]:
"""初始化"""

import pandas as pd
import numpy as np

stocks = pd.read_csv('csv/stocks-cny.csv', dtype={'Symbol':str})
trades = pd.read_csv('csv/trades-cny.csv', dtype={'Symbol':str, 'Qty':np.int64})
dividends = pd.read_csv('csv/dividends-cny.csv', dtype={'Symbol':str, 'Qty':np.int64})

In [2]:
def get_summary(stocks, trades, divids):
    '''
    stocks: A pandas dataframe that lists all traded stocks.
    trades: A pandas dataframe that lists all trades.
    dividends: A pandas dataframe that lists all dividends.

    Return a dataframe that includes all useful trade indicators.
    For example, Total buy/sell amount for a stock and average buy/sell price.
    '''

    # Set stock's symbol as Dataframe index.
    stocks.set_index('Symbol', inplace=True)
    trades.set_index('Symbol', inplace=True)
    divids.set_index('Symbol', inplace=True)

    # Calculate actual amount for each trade.
    proceed = trades['Price'] * trades['Qty'].abs()
    fee = trades['Commission'] + trades['Tax']
    trades['Basis'] = np.where( trades['Transaction']=='BUY',
            proceed+fee, proceed-fee)

    # Generate Pandas groupby object for trades and dividends.
    group_trade = trades.groupby([trades.index, trades['Transaction']])
    group_divid = divids.groupby(divids.index)

    # Claculate total bought/sold qty for each stock.
    qty_sum = group_trade['Qty'].sum().unstack(fill_value=0)
    stocks_detail = stocks.join(qty_sum)
    stocks_detail.columns = ['Name', 'B_Qty', 'S_Qty']
    stocks_detail['Qty'] = stocks_detail['B_Qty'] + stocks_detail['S_Qty']

    # Calculate average bought/sold price.
    basis = group_trade['Basis'].sum().unstack(fill_value=0)
    stocks_detail['B_Cost'] = basis['BUY']/stocks_detail['B_Qty']
    stocks_detail['S_Cost'] = basis['SELL']/stocks_detail['S_Qty'].abs()

    # Calculate realized profit/loss
    sold_cost = stocks_detail['B_Cost'] * stocks_detail['S_Qty'].abs()
    stocks_detail['R_PnL'] = basis['SELL']- sold_cost

    # Calculate dividend for each stock.
    divid_fee = divids['Commission'] + divids['Tax']
    divids['Dividend'] = divids['PerShare']*divids['Qty'] - divid_fee
    stocks_detail['Dividend'] = group_divid['Dividend'].sum()

    # Fill NaN with zero value for realized profit/loss and dividend.
    stocks_detail['R_PnL'].fillna(0.0, inplace=True)
    stocks_detail['Dividend'].fillna(0.0, inplace=True)

    #return stocks_detail.reset_index().round(2)
    return stocks_detail.round(2)

In [3]:
"""测试"""

stocks_summary = get_summary(stocks, trades, dividends)
stocks_summary.head(10)

,Name,B_Qty,S_Qty,Qty,B_Cost,S_Cost,R_PnL,Dividend
Symbol,,,,,,,,
510900,H股ETF,140000,-140000,0,0.97,0.87,-14488.12,0.00
150176,H股B,218300,-218300,0,0.71,0.70,-2502.94,0.00
159920,恒生ETF,180000,-180000,0,1.12,1.12,1103.15,0.00
600519,贵州茅台,650,-650,0,215.15,312.23,63098.15,3009.89
600036,招商银行,8000,-8000,0,17.99,16.12,-14976.55,-167.50
600660,福耀玻璃,7000,-4000,3000,17.22,20.51,13141.31,5175.00
600886,国投电力,22000,-22000,0,6.82,6.56,-5595.10,2519.06
000651,格力电器,7000,-7000,0,21.42,23.21,12526.62,10050.00
000002,万科A,3000,-3000,0,24.43,19.08,-16061.62,0.00


In [4]:
"""定义获取股票最新报价的函数"""

import datetime
import pandas_datareader.data as web
import tushare as ts

def getYahooQuote(symbols):
    start = datetime.datetime.today()
    end = start
    
    quotes_dict = {}
    yahoo_symbols = symbols + '.HK'
    pnl = web.DataReader(yahoo_symbols, 'yahoo', start, end)
    df = pnl['Close'].T
    df.index.name = 'Symbol'
    df.columns=['Last']
    df.index = df.index.map(lambda x: x.rstrip('.HK'))
    
    return df


def get_ts_quote(symbols, isA=True):
    cons = ts.get_apis()
    last = datetime.datetime.today() - datetime.timedelta(days=7)
    
    quotes_dict = {}
    for item in symbols:
        if isA:
            df = ts.bar(item, conn=cons, freq='D',
                start_date=last, end_date='')
        else:
            df = ts.bar(item, conn=cons, asset='X',
                start_date=last, end_date='')
        quotes_dict[item] = df['close'].iloc[0]

    quotes = pd.Series(quotes_dict)
    return quotes

In [5]:
"""为当前持仓股票获取最近价格"""

stocks_hold = stocks_summary.loc[lambda df: df.Qty > 0]
stock_last = get_ts_quote(stocks_hold.index)
stocks_summary['Last'] = stock_last

In [6]:
"""为未平仓股票计算未实现盈亏"""

stocks_summary['UR_PnL'] = stocks_summary['Qty'] * (stocks_summary['Last'] - stocks_summary['B_Cost'])
stocks_summary['Earning'] = stocks_summary['Dividend'] + stocks_summary['R_PnL'] + stocks_summary['UR_PnL']
stocks_summary.head(10)

,Name,B_Qty,S_Qty,Qty,B_Cost,S_Cost,R_PnL,Dividend,Last,UR_PnL,Earning
Symbol,,,,,,,,,,,
510900,H股ETF,140000,-140000,0,0.97,0.87,-14488.12,0.00,NaN,NaN,NaN
150176,H股B,218300,-218300,0,0.71,0.70,-2502.94,0.00,NaN,NaN,NaN
159920,恒生ETF,180000,-180000,0,1.12,1.12,1103.15,0.00,NaN,NaN,NaN
600519,贵州茅台,650,-650,0,215.15,312.23,63098.15,3009.89,NaN,NaN,NaN
600036,招商银行,8000,-8000,0,17.99,16.12,-14976.55,-167.50,NaN,NaN,NaN
600660,福耀玻璃,7000,-4000,3000,17.22,20.51,13141.31,5175.00,28.750,34590.0,52906.31
600886,国投电力,22000,-22000,0,6.82,6.56,-5595.10,2519.06,NaN,NaN,NaN
000651,格力电器,7000,-7000,0,21.42,23.21,12526.62,10050.00,NaN,NaN,NaN
000002,万科A,3000,-3000,0,24.43,19.08,-16061.62,0.00,NaN,NaN,NaN
